In [6]:
from hydradx.model.amm.omnipool_amm import OmnipoolState
from hydradx.model.amm.agents import Agent
from hydradx.model.processing import parse_kraken_orderbook
from hydradx.model.amm.arbitrage_agent import get_arb_swaps, calculate_profit, execute_arb


import json, os


In [7]:
f = open('data/hdxusd_orderbook.json', 'r')
y = json.loads(f.read())
hdxusd_orderbook = y['result']['HDXUSD']

f = open('data/dotusd_orderbook.json', 'r')
y = json.loads(f.read())
dotusd_orderbook = y['result']['DOTUSD']



In [8]:

# dot_usdt_order_book = {
#     'bids': [{'price': 3.60, 'amount': 200},
#              {'price': 3.59, 'amount': 100},
#              {'price': 3.50, 'amount': 100},
#              {'price': 3.40, 'amount': 2000}],
#     'asks': [{'price': 3.70, 'amount': 100},
#              {'price': 3.74, 'amount': 5000},
#              {'price': 3.80, 'amount': 200},
#              {'price': 3.90, 'amount': 2000}]
# }
# 
# hdx_usdt_order_book = {
#     'bids': [{'price': 0.03, 'amount': 2000},
#              {'price': 0.025, 'amount': 2000},
#              {'price': 0.02, 'amount': 2000},
#              {'price': 0.015, 'amount': 2000}],
#     'asks': [{'price': 0.04, 'amount': 2000},
#              {'price': 0.05, 'amount': 2000},
#              {'price': 0.06, 'amount': 2000},
#              {'price': 0.07, 'amount': 2000}]
# }

# hdx_dot_order_book = {
#     'bids': [{'price': 0.005, 'amount': 2000},
#              {'price': 0.004, 'amount': 2000}],
#     'asks': [{'price': 0.0052, 'amount': 2000},
#              {'price': 0.0055, 'amount': 2000}]
# }

dot_usdt_order_book = parse_kraken_orderbook(dotusd_orderbook)
hdx_usdt_order_book = parse_kraken_orderbook(hdxusd_orderbook)

order_book = {
    ('DOT','USDT'): dot_usdt_order_book,
    ('HDX','USDT'): hdx_usdt_order_book,
    # ('HDX','DOT'): hdx_dot_order_book
}

tokens = {
    'USDT': {
        'liquidity': 2062772,
        'LRNA': 2062772
    },
    'DOT': {
        # 'liquidity': 389000,
        'liquidity': 350000,
        'LRNA': 1456248
    },
    'HDX': {
        'liquidity': 108000000,
        'LRNA': 494896
    }
}

lrna_fee = 0.0005
asset_fee = 0.0025
cex_fee = 0.0016

# lrna_fee = 0.0
# asset_fee = 0.0
# cex_fee = 0.0

op_state = OmnipoolState(
            tokens=tokens,
            lrna_fee=lrna_fee,
            asset_fee=asset_fee,
    preferred_stablecoin='USDT',
        )

init_state = op_state.copy()


In [9]:
all_swaps = get_arb_swaps(op_state, order_book, lrna_fee, asset_fee, cex_fee)
print(all_swaps)

3.7604754221748347
0.00458237037037037
{('DOT', 'USDT'): [('sell', {'price': 3.7398, 'amount': 14.061}), ('sell', {'price': 3.7406, 'amount': 93.629}), ('sell', {'price': 3.7407, 'amount': 43.0}), ('sell', {'price': 3.7408, 'amount': 40.0}), ('sell', {'price': 3.741, 'amount': 390.018}), ('sell', {'price': 3.7417, 'amount': 3957.426}), ('sell', {'price': 3.7418, 'amount': 3334.869}), ('sell', {'price': 3.7429, 'amount': 374.508}), ('sell', {'price': 3.743, 'amount': 1347.0}), ('sell', {'price': 3.7432, 'amount': 1055.139461517334})], ('HDX', 'USDT'): []}


In [10]:
state = init_state.copy()
init_agent = Agent(holdings={'USDT': 1000000000, 'DOT': 1000000000, 'HDX': 1000000000}, unique_id='bot')
agent = init_agent.copy()
execute_arb(state, agent, all_swaps, cex_fee=cex_fee)

profit = calculate_profit(init_agent, agent)

print(profit)
            
# for tkn_pair in all_swaps:
#     print(tkn_pair)
#     op_spot = OmnipoolState.price(state, tkn_pair[0], tkn_pair[1])
#     buy_spot = op_spot / ((1 - lrna_fee)*(1 - asset_fee))
#     sell_spot = op_spot * (1 - lrna_fee) * (1 - asset_fee)
#     print((sell_spot, op_spot, buy_spot))
    


{'USDT': 2083.132005214691, 'DOT': 0.0, 'HDX': 0}
